**`Webinar 2024-04-17`**

[YouTube Recording](https://www.youtube.com/watch?v=TSbiawPaQcQ&list=PLePKvgYhNOFxg3O7gXfmxaKp0EWXEhvNC&index=11)

# Book Ideation and Generation using Document Review and Long Form

### Overview

The goal is to write a quality educational book in the simplest way possible. We will go through the ideation phase to set the tone and topics of the book.

##### Project Flow

1. Create knowledge bases for each chapter we want in the book.
2. Get some user inputs in regard to overall theme and style.
3. For each chapter knowledge base, run a document review where the "final review" output is a chapter outline.
4. Feed each of the document review outlines into a long form generation call.

##### Initialization

##### `NOTE: To run this, you will need to replace with your own chapter KBs as well as book themes/guidance in the next 2 cells`

In [1]:
import superpowered
import requests
import time
import tqdm

from IPython.display import display, Markdown

SP_BASE_URL = 'https://api.superpowered.ai/v1'
SP_API_KEY_ID = 'server_539544f6a93253bbcb3e24fe8ec90cde'
SP_API_KEY_SECRET = 'KDBjujcH2kAwi5TMHC3jqplPOlfuhL3F1LB4kHa9YUY'
API_AUTH = (SP_API_KEY_ID, SP_API_KEY_SECRET)
superpowered.set_api_key(SP_API_KEY_ID, SP_API_KEY_SECRET)

knowledge_base_ids = [
    '08844d05-4e09-479d-9770-6faebccfceb6',  # STOICISM
    'a0fd538c-06ec-43ae-849b-f9570caca2c6',  # EPICUREANISM
    '87401ada-dfcb-4e3a-afbc-8fe9ead62bb4',  # MINIMALISM
    '6fde0338-0856-477b-bfa0-439432dfbcd1',  # HEDONISM
    'a78019b4-c498-412b-9daf-737698fab4c5',  # EXISTENTIALISM
    '740b2668-ecab-4372-9bc4-6a01a4f88560',  # NIHILISM
    '183cee2f-2053-480b-8e86-9377489208c5',  # PRAGMATISM
    '0e75e305-b314-40a3-a4fb-7f357bda0c10',  # IDEALISM
]


##### User input describing goal for the book

In [26]:
BOOK_DIRECTORY = 'living_a_good_life'
BOOK_TITLE = 'Living a Good Life in the Age of AI and Automation'
BOOK_THEME = 'The general theme of this book is to look at what it means to live a good life in the context of stoicism, epicureanism, minimalism, hedonism, existentialism, nihilism, pragmatism, and idealism. Each chapter will cover one of these 8 philosophical schools of thought.'
BOOK_OVERALL_STYLE_GUIDANCE = 'The style should be widely accessible to the general public with a focus on how each of the philosophies are relevant in the modern age of AI and automation. It is important to use a variable vocabulary and make sure that nothing is repetitive and dry. Be fun and present ideas with examples. Avoid bland closing statements like "in summary", "in conclusion", etc. The book should be a fun and engaging read.'
BOOK_CHAPTER_STYLE_GUIDANCE = 'Each chapter should start with a general overview of the philosophy in question and explain how the concepts are relevant in the modern age of AI and automation. Give examples of what living by this philosophy would look like in scenarios like: "remote work and lack of social interacation", "complete automation with no jobs", "human population expantion", "virtual reality", "augmented reality", "cell phones", "immersive video games", etc.'

STYLE_SAMPLE = """
I can remember starving in a
small room in a strange city
shades pulled down, listening to
classical music
I was young I was so young it hurt like a knife
inside
because there was no alternative except to hide as long
as possible—
not in self-pity but with dismay at my limited chance:
trying to connect.

the old composers — Mozart, Bach, Beethoven,
Brahms were the only ones who spoke to me and
they were dead.

finally, starved and beaten, I had to go into
the streets to be interviewed for low-paying and
monotonous
jobs
by strange men behind desks
men without eyes men without faces
who would take away my hours
break them
piss on them.

now I work for the editors the readers the
critics

but still hang around and drink with
Mozart, Bach, Brahms and the
Bee
some buddies
some men
sometimes all we need to be able to continue alone
are the dead
rattling the walls
that close us in."""

EXCLUDE_WORDS_AND_PHRASES = [
    'journey',
    'delve',
    'navigate',
    'realm',
    'critical',
    'overstated',
    'in conclusion',
    'in summary',
    'in closing',
]

##### Helper functions that will run document reviews and long form plus stream results

- `run_and_display_review()` function will start a document review, show an active reading progress bar and then stream final review results as they happen.
- `submit_long_form_job()` just submits a long form job so we can generate all the chapters in parallel.


In [27]:
def run_and_display_review(kb: dict, ar_instructions: str, fr_instructions: str):
    url = f'{SP_BASE_URL}/knowledge_bases/{kb["id"]}/review'
    payload = {
        'active_reading_instructions': ar_instructions,
        'final_review_instructions': fr_instructions,
    }
    resp = requests.post(url, json=payload, auth=API_AUTH)
    final_review_display = display(Markdown('Review pending...'), display_id=True)
    progress_bar = tqdm.tqdm(total=1)
    progress_bar.set_description(f'Active Reading Progress for {kb["title"]}')

    # keep request job until it's completed (or failed)
    while resp.json()['status'] not in ['COMPLETE', 'FAILED']:
        resp = requests.get(resp.json()['status_url'], auth=API_AUTH)
        progress_bar.n = resp.json()['response']['active_reading_progress_pct']
        if progress_bar.n >= 1:
            final_review_display.update(Markdown(resp.json()["response"]["final_review"]))
        progress_bar.refresh()

    progress_bar.close()
    return resp.json()['response']['final_review']


def submit_long_form_job(kb: dict, prompt: str):
    url = f'{SP_BASE_URL}/long_form'
    payload = {
        'knowledge_base_ids': [kb['id']],
        'model': 'gpt-4',
        'response_length': 'long',
        'prompt': prompt,
        'use_web_search': True,
    }
    resp = requests.post(url, json=payload, auth=API_AUTH)
    return resp.json()['id']


def revise_long_form_section(text_to_revise: str, job_id: str, section_number: int):
    url = f'{SP_BASE_URL}/revise'
    payload = {
        'input': text_to_revise,
        'style_sample': STYLE_SAMPLE,
        'exclude_words': EXCLUDE_WORDS_AND_PHRASES,
    }
    response = requests.post(url, auth=API_AUTH, json=payload)
    revision_output_display = display(Markdown('Revision pending...'), display_id=True)

    while response.json()['status'] not in ['COMPLETE', 'FAILED']:
        response = requests.get(response.json()['status_url'], auth=API_AUTH)
        if response.json()['response']['output']:
            revision_output_display.update(Markdown(response.json()['response']['output']))

    return response.json()['response']['output']



##### Loop through each knowledge base and run the document review

In [4]:
active_reading_instructions = f"""\
You are reading through a knowledge base on {{philosophy}}. Please extract information that is relevant to living a good life.

The infomration that is extracted will be used to write a chapter about {{philosophy}} for a book with the following theme: {BOOK_THEME}

Your output should be a list of key points and examples that could be relevant to the modern age of AI and automation. The information you are reading through might not directly mention AI and automation so it's up to you to decide if the information is relevant or not.
"""

final_review_instructions = f"""\
Given the list key points and examples from the active reading, please generate a chapter outline. The book must follow these style guidelines:

1. {BOOK_OVERALL_STYLE_GUIDANCE}
2. {BOOK_CHAPTER_STYLE_GUIDANCE}

The chapter outline should contain a list of detailed section descriptions. Please try to keep the number of sections between 5 and 7
"""

outlines = []
for kb_id in knowledge_base_ids:
    # get the knowledge base object
    kb_url = f'{SP_BASE_URL}/knowledge_bases/{kb_id}'
    resp = requests.get(kb_url, auth=API_AUTH)
    kb = resp.json()

    outline = run_and_display_review(kb, active_reading_instructions.format(philosophy=kb['title']), final_review_instructions)
    outlines.append((kb, outline))

**Chapter Outline: Stoicism and Modern Life in the Age of AI and Automation**

**1. Introduction to Stoicism: Foundations and Key Concepts**
   - Brief history of Stoicism, from Zeno of Citium to its influence on Roman thinkers like Marcus Aurelius.
   - Explanation of the core Stoic beliefs: virtue as the sole good, the role of eudaimonia (flourishing), and the importance of living in accordance with nature.
   - Introduction to the Stoic virtues: wisdom, courage, justice, and temperance.

**2. Stoicism and the Digital Age: Virtue in Virtual Realities**
   - Exploration of how Stoic virtues can guide behavior in virtual environments, such as immersive video games and virtual reality.
   - Discussion on maintaining Stoic equanimity and ethical engagement in increasingly digital social interactions.
   - Examples of applying Stoic indifference to preferred and dispreferred "indifferents" in the context of digital wealth and online status.

**3. Stoicism and AI: Navigating New Frontiers of Control**
   - Analysis of Stoic teachings on focusing on what is within our control in the face of AI-driven automation and decision-making.
   - The role of Stoic acceptance in adapting to job displacement due to automation.
   - How Stoic virtues can help maintain human dignity and ethical considerations in AI development and implementation.

**4. Stoicism and Environmental Challenges: A Stoic Response to Global Issues**
   - Application of Stoic principles to modern challenges such as climate change, resource scarcity, and sustainable development.
   - Discussion on Stoic cosmopolitanism and its implications for global justice and equity in resource distribution.
   - Stoic views on the natural world and the importance of living harmoniously within it, relevant to environmental education.

**5. Stoicism in Personal Development: From Theory to Practice**
   - Practical advice on incorporating Stoic exercises into daily life to build resilience, self-awareness, and emotional intelligence.
   - The role of modern Stoicism in mental health, particularly its connections with Cognitive Behavioral Therapy (CBT) and Rational Emotive Behavior Therapy (REBT).
   - Case studies of individuals applying Stoic principles to overcome personal adversity and improve their quality of life.

**6. The Stoic Community: Revival and Relevance in the 21st Century**
   - Overview of the modern Stoicism movement, including its presence in social media, blogs, and academic discussions.
   - Contributions of contemporary philosophers to the revival of Stoicism and its adaptation to modern contexts.
   - Future prospects of Stoicism as a living philosophy in a rapidly changing world.

**7. Conclusion: Stoicism as a Guide for the Modern Age**
   - Recapitulation of how Stoicism provides tools for dealing with the complexities of modern life, including technology and environmental issues.
   - Final thoughts on the enduring relevance of Stoic philosophy in promoting a life of virtue, resilience, and harmony with the universe.

This outline aims to present Stoicism not just as an ancient philosophy but as a vibrant and practical approach to modern challenges, particularly in the context of technological advancement and environmental concerns. Each section will blend historical insights with contemporary applications, making Stoicism accessible and relevant to a broad audience.

Active Reading Progress for STOICISM [CH. 1]: 100%|██████████| 1/1 [01:53<00:00, 113.91s/it]      


**Chapter Outline: Epicureanism in the Age of AI and Automation**

**1. Introduction to Epicureanism: Foundations and Key Concepts**
   - Brief history of Epicureanism, founded by Epicurus in 307 BCE.
   - Core principles: seeking modest, sustainable pleasure and freedom from fear.
   - Distinction between Epicureanism and common hedonism.
   - Introduction to the concepts of aponia (absence of pain) and ataraxia (freedom from disturbance).

**2. Epicurean Philosophy and Modern Technology**
   - Exploration of how Epicurean principles apply to modern contexts like AI and automation.
   - Discussion on the Epicurean pursuit of tranquility in the digital age, amidst the noise of constant connectivity.
   - The role of technology in either enhancing or detracting from sustainable pleasure and tranquility.

**3. Epicureanism and Social Dynamics in the Digital Era**
   - Analysis of Epicurean withdrawal from politics and its relevance to social media and online interactions.
   - The value of friendship in Epicureanism and its interpretation in an age of digital communication.
   - How virtual and augmented realities could serve as tools for achieving Epicurean goals or pitfalls that lead to pain and disturbance.

**4. The Epicurean Response to Automation and Joblessness**
   - Discussion on how Epicureanism would approach the concept of a jobless future due to complete automation.
   - Strategies for maintaining mental tranquility and pleasure without the traditional structure of employment.
   - The importance of limiting desires in a world where material abundance is facilitated by AI and automation.

**5. Epicurean Ethics and the Pursuit of Pleasure in a High-Tech World**
   - Examination of Epicurean ethics in the context of modern dilemmas posed by technology, such as privacy concerns and data ethics.
   - The balance between sensory pleasures and intellectual pleasures in an era dominated by immersive video games and virtual realities.
   - How Epicureanism can guide a balanced use of technology, avoiding overindulgence and maintaining mental health.

**6. Longevity, Death, and the Epicurean Perspective**
   - Epicurus' teachings on the naturalness and inevitability of death, and its implications for modern society's quest for longevity and immortality through technology.
   - Discussion on how AI and genetic engineering might challenge or reinforce Epicurean views on death and the fear of gods.
   - The role of science and philosophy in alleviating fears that technology might exacerbate.

**7. Conclusion: Embracing Epicureanism in the Technological Landscape**
   - Summarizing how Epicureanism can offer valuable insights into living a good life amidst rapid technological advancements.
   - Potential for an Epicurean revival in contemporary society as a counterbalance to the excesses of modern technology.
   - Encouragement for readers to explore how they can apply Epicurean principles to navigate the complexities of modern life.

This chapter will weave together historical insights with contemporary examples, illustrating how the ancient philosophy of Epicureanism remains relevant and can provide guidance in the age of AI and automation. The focus will be on making the ideas accessible and engaging, using modern scenarios to demonstrate the practical application of Epicurean principles.

Active Reading Progress for EPICUREANISM [CH. 2]: 100%|██████████| 1/1 [01:08<00:00, 68.01s/it]      


**Chapter Outline: Minimalism and the Good Life in the Age of AI and Automation**

**1. Introduction to Minimalism**
   - Define minimalism in both historical and contemporary contexts.
   - Discuss the evolution from a visual and architectural movement to a lifestyle philosophy.
   - Introduce the relevance of minimalism in the modern age, particularly in relation to AI and automation.

**2. Minimalism in Daily Life and Technology**
   - Explore how minimalism can be applied to modern technology usage, such as smartphones, digital clutter, and social media.
   - Discuss the minimalist approach to digital life, focusing on essential apps and tools, and decluttering digital spaces.
   - Example scenarios: Managing notifications to reduce stress, using minimalist design principles in app development.

**3. Minimalism in Work and Automation**
   - Analyze the impact of minimalism in work environments, especially with the rise of remote work and automation.
   - Discuss how minimalism can help in coping with job automation, by focusing on essential skills and simplifying work processes.
   - Example scenarios: Designing minimalist workspaces at home, prioritizing tasks in a highly automated job environment.

**4. Environmental and Urban Minimalism**
   - Discuss the role of minimalism in addressing environmental issues and urban overcrowding.
   - Explore minimalist architecture as a response to chaotic urban life and its principles in creating sustainable living spaces.
   - Example scenarios: Minimalist home designs that reduce environmental impact, city planning that incorporates minimalist principles.

**5. Psychological and Spiritual Benefits of Minimalism**
   - Examine the psychological impacts of adopting a minimalist lifestyle, including reduced stress and increased focus.
   - Discuss the spiritual dimensions of minimalism, particularly its roots in Zen philosophy and how these can be applied to modern life challenges.
   - Example scenarios: Using minimalism to enhance mindfulness practices, simplifying life to cope with the fast pace of modern society.

**6. Challenges and Critiques of Minimalism**
   - Address common challenges faced when adopting a minimalist lifestyle, such as resistance from peers or the initial increase in stress.
   - Critique the potential downsides of minimalism, including over-simplification and the risk of ignoring necessary complexities of life.
   - Example scenarios: Navigating family life when only one member is pursuing minimalism, the balance between minimalism and cultural expectations of success.

**7. Future of Minimalism in an AI-driven World**
   - Speculate on the future role of minimalism as AI and automation become increasingly prevalent in everyday life.
   - Discuss how minimalism might evolve as a philosophy and practice in response to advancements in technology and changes in societal norms.
   - Example scenarios: Minimalism in virtual reality settings, the role of minimalism in designing user interfaces for AI systems.

This chapter will not only define and explore minimalism but will also connect it dynamically with contemporary issues and technological advancements, making it relevant and engaging for readers interested in understanding how ancient philosophies can guide modern lives.

...

Active Reading Progress for NIHILISM [CH.6]: 100%|██████████| 1/1 [01:16<00:00, 76.44s/it]      


**Chapter Outline: Pragmatism and Living a Good Life in the Age of AI and Automation**

**1. Introduction to Pragmatism**
   - Brief history of pragmatism, highlighting its origins in the United States in the late 19th century.
   - Introduction to key figures such as Charles Sanders Peirce, William James, and John Dewey.
   - Explanation of the core principles of pragmatism: the truth of a belief is determined by its practical consequences and utility.

**2. Pragmatism in the Modern World: AI and Automation**
   - Discussion on how pragmatism's focus on practical consequences and problem-solving is crucial in the era of AI and automation.
   - Examples of how pragmatic approaches can guide ethical AI development, focusing on outcomes rather than theoretical purity.
   - Exploration of pragmatism in automation policies, emphasizing adaptable and beneficial outcomes for society.

**3. Pragmatism and Daily Technology: From Smartphones to Virtual Reality**
   - Analysis of how everyday technologies like smartphones and VR can be understood through a pragmatic lens.
   - Discussion on the practical impacts of these technologies on human behavior and societal norms.
   - Insights into how pragmatism advocates for technology that enhances human experience and solves real-world problems.

**4. Pragmatic Approaches to Global Challenges: Climate Change and Urbanism**
   - Application of pragmatism to large-scale issues like climate change, emphasizing practical steps and adaptable policies.
   - Exploration of urbanism through a pragmatic viewpoint, focusing on creating livable and sustainable cities through actionable insights.
   - Case studies of pragmatic policies in action, demonstrating their effectiveness in addressing complex global issues.

**5. The Role of Pragmatism in Personal and Professional Growth**
   - Discussion on how pragmatism can influence personal decision-making and career development in a fast-changing world.
   - Examples of pragmatic thinking in personal growth, such as continuous learning and adapting to new skills.
   - Insights into how pragmatism can lead to more fulfilling and effective professional practices.

**6. Pragmatism and Social Interactions in the Digital Age**
   - Examination of how pragmatism can help navigate the complexities of social interactions in an increasingly digital world.
   - Discussion on the role of pragmatism in understanding and improving online communication and community building.
   - Analysis of pragmatic approaches to digital etiquette and the maintenance of meaningful human connections despite technological barriers.

**7. Conclusion: Embracing Pragmatism for a Better Future**
   - Summary of how embracing pragmatism can lead to practical and beneficial outcomes in various aspects of life.
   - Encouragement for readers to apply pragmatic principles to tackle personal and societal challenges.
   - Final thoughts on the potential of pragmatism to adapt and thrive in the future, particularly in relation to ongoing technological advancements.

This chapter aims to not only introduce readers to the foundational concepts of pragmatism but also to demonstrate its relevance and application in contemporary issues shaped by technology and rapid societal changes. By providing real-world examples and actionable insights, the chapter will engage readers in a thoughtful exploration of how pragmatism can contribute to living a good life in the modern age.

Active Reading Progress for PRAGMATISM [CH. 7]: 100%|██████████| 1/1 [01:41<00:00, 101.53s/it]      


**Chapter Outline: Idealism and the Good Life in the Age of AI and Automation**

**1. Introduction to Idealism: Mind Over Matter**
   - Briefly introduce the concept of idealism, emphasizing its assertion that reality is fundamentally mental or spiritual.
   - Discuss the historical roots of idealism, from ancient philosophies like Plato’s theory of forms to modern interpretations in the context of AI and automation.
   - Set the stage for exploring how idealism can guide us in navigating a world increasingly dominated by technology.

**2. The Idealist View of Reality in a Digitally Dominated World**
   - Explain metaphysical and epistemological idealism, highlighting differences and implications for understanding reality.
   - Discuss how idealism intersects with modern challenges such as virtual reality, augmented reality, and the digital transformation of social interactions.
   - Use examples like immersive video games and virtual workspaces to illustrate idealist perspectives on digital experiences as genuine forms of reality.

**3. Living the Good Life Through Idealism Amidst Automation**
   - Explore the implications of a fully automated society where jobs are scarce, focusing on the idealist view that reality and experiences are mentally constructed.
   - Discuss how idealism can provide a framework for finding meaning and purpose in life beyond traditional employment.
   - Introduce concepts like the role of creativity and imagination in crafting fulfilling lives when traditional roles are obsolete.

**4. Idealism, Self-Perception, and Social Expansion**
   - Analyze the role of idealism in shaping our self-perception and interactions in an expanding human population and increasingly interconnected world.
   - Discuss the potential for idealism to foster a sense of global community and empathy, crucial in a world facing large-scale social and environmental challenges.
   - Provide examples of how idealist philosophy can influence ethical behavior and social responsibility in the age of AI.

**5. The Ethical Implications of Idealism in Technology and AI**
   - Delve into the ethical considerations of AI development and deployment from an idealist standpoint, focusing on consciousness and moral responsibility.
   - Discuss the potential for AI to reflect human consciousness and the idealist view that reality is a mental construct, raising questions about the rights and treatment of AI entities.
   - Explore scenarios where AI could potentially contribute to or hinder the achievement of an idealist good life.

**6. Future Visions: Idealism in the Advancement of Human Consciousness**
   - Speculate on the future role of idealism in guiding human evolution, particularly as it relates to enhancing or merging human consciousness with AI.
   - Discuss the philosophical implications of technologies like brain-computer interfaces and AI-enhanced cognitive abilities from an idealist perspective.
   - Consider the potential for a new form of society where the lines between mind and machine are blurred, guided by idealist principles.

**7. Conclusion: Embracing Idealism in the Modern World**
   - Recapitulate the key points discussed in the chapter, emphasizing the relevance of idealism in helping us navigate the challenges and opportunities presented by AI and automation.
   - Encourage readers to consider how adopting an idealist perspective might change their personal and collective approach to technology and life.
   - End with a reflective note on the enduring power of ideas and consciousness in shaping our reality, urging a thoughtful integration of idealism in daily life and policy.

This chapter will engage readers by connecting ancient philosophical insights with contemporary issues, using relatable examples and thought-provoking scenarios to illustrate how idealism remains a vital and relevant philosophy in the age of technology.

Active Reading Progress for IDEALISM [CH. 8]: 100%|██████████| 1/1 [02:54<00:00, 174.58s/it]      


##### Create the prompt for each chapter with a little semantic sugar around the outlines created by the document review step and submit long form job

In [14]:
chapter_ids = []

for kb, outline in outlines:
    prompt = f"""\
    You have been assigned to write a chapter in a book titled "{BOOK_TITLE}" about {kb['title']}. Please follow this outline when writing the chapter:

    {outline}

    {BOOK_OVERALL_STYLE_GUIDANCE}
    {BOOK_CHAPTER_STYLE_GUIDANCE}

    DO NOT EVER USE CLOSINGS LIKE "in conclusion", "in summary", "ultimately", etc. They are boring and you should have more pride in your output.

    When you generate a title, please include the chapter number and the name of the philosophy. This chapter is {kb['title']}".
    """
    print('STARTING CHAPTER:', kb['title'])
    chapter_job_id = submit_long_form_job(kb, prompt)
    chapter_ids.append(chapter_job_id)
    

STARTING CHAPTER: STOICISM [CH. 1]
STARTING CHAPTER: EPICUREANISM [CH. 2]
STARTING CHAPTER: MINIMALISM [CH. 3]
STARTING CHAPTER: HEDONISM [CH. 4]
STARTING CHAPTER: EXISTENTIALISM [CH. 5]
STARTING CHAPTER: NIHILISM [CH.6]
STARTING CHAPTER: PRAGMATISM [CH. 7]
STARTING CHAPTER: IDEALISM [CH. 8]


# START OF PART 2 --- 2024-02-21

- Edit the title and export each chapter as a markdown file and put in the book folder.
- Generate an "Introduction" and "Conclusion" chapter by running a document review on a knowledge base with all the outlines and generated chapters. These reviews will gnerate the outlines and then be fed into the `long_form` endpoint.
- Use pandoc to export the .epub file.

##### Edit titles so they are simply the name of the philosophy

In [17]:
def patch_chapter_title(new_title, job_id):
    url = f'{SP_BASE_URL}/long_form/{job_id}'
    resp = requests.patch(url, json={'title': new_title}, auth=API_AUTH)
    return resp.json()

for (kb, _), chapter_job_id in zip(outlines, chapter_ids):
    # extract philosophy in question from the KB title and make that the chapter title
    philosophy = kb['title'].split(' ')[0].capitalize()
    print(f'PATCHING CHAPTER: {philosophy}')
    resp = patch_chapter_title(philosophy, chapter_job_id)


PATCHING CHAPTER: Stoicism
PATCHING CHAPTER: Epicureanism
PATCHING CHAPTER: Minimalism
PATCHING CHAPTER: Hedonism
PATCHING CHAPTER: Existentialism
PATCHING CHAPTER: Nihilism
PATCHING CHAPTER: Pragmatism
PATCHING CHAPTER: Idealism


##### Export each chapter into markdown format and read the contents

In [18]:
# helper function to get the markdown of a job
def get_job_markdown(job_id):
    url = f'{SP_BASE_URL}/long_form/{job_id}/exports/text'
    resp = requests.get(url, auth=API_AUTH)
    download_url = resp.json()['download_url']
    return requests.get(download_url).text

chapter_markdown_contents = []
for chapter_job_id in chapter_ids:
    markdown = get_job_markdown(chapter_job_id)
    chapter_markdown_contents.append(markdown)

##### Create a new knowledge base for the all of the content generated up to this point

In [9]:
book_content_kb = superpowered.create_knowledge_base(
    title='Outlines and Chapters for "Living a Good Life in the Age of AI and Automation"',
)

# create documents from the text generated in the previous steps
for (kb, outline), chapter_markdown in zip(outlines, chapter_markdown_contents):
    # create a document for the outline
    superpowered.create_document_via_text(
        knowledge_base_id=book_content_kb['id'],
        title=f'Chapter Outline for "{kb["title"]}"',
        content=outline,
        auto_context=True,
    )
    # create a document for the chapter
    superpowered.create_document_via_text(
        knowledge_base_id=book_content_kb['id'],
        title=f'Chapter Content for "{kb["title"]}"',
        content=chapter_markdown,
        auto_context=True,
    )


# list documents in the knowledge base
total_docs = len(superpowered.list_documents(book_content_kb['id']))
completed_docs = 0
while completed_docs < total_docs:
    completed_docs = len(superpowered.list_documents(book_content_kb['id'], vectorization_status='COMPLETE'))
    print(f'{completed_docs}/{total_docs} documents completed...')
    time.sleep(5)

0/16 documents completed...
0/16 documents completed...
6/16 documents completed...
12/16 documents completed...
16/16 documents completed...


##### Run a document review to generate an outline for the introduction and conclusion

In [19]:
active_reading_instructions = f"""\
You are reading through a knowledge base that contains the contents for a book about living a good life in the age of AI and automation.

Each chapter in the book tackles this question from the perspective of a different philosophical school of thought.

Here are the chapters of the book: {', '.join([f'- {kb["title"]}' for kb, _ in outlines])}

Please extract information that is relevant to living a good life in the age of AI and automation. The information that is extracted will be used to write a summary for the book.
"""

introduction_final_review_instructions = f"""\
Your task is to write an outline for an introduction that introduces the book and the general theme of the book. The introduction should be engaging and set the stage for the rest of the book.

Break your introduction into 3-5 sections. Each section should encompass themes from all the chapters in the book rather than focus on a single chapter.

{BOOK_OVERALL_STYLE_GUIDANCE}
"""


conclusion_final_review_instructions = f"""\
Your task is to write an outline for a conclusion that summarizes the book and the general theme of the book. The conclusion should be engaging and leave the reader with a sense of closure.

Break your conclusion into 3-5 sections. Each section should encompass themes from all the chapters in the book rather than focus on a single chapter.

{BOOK_OVERALL_STYLE_GUIDANCE}
"""


# run the document reviews
introduction_outline = run_and_display_review(book_content_kb, active_reading_instructions, introduction_final_review_instructions)
conclusion_outline = run_and_display_review(book_content_kb, active_reading_instructions, conclusion_final_review_instructions)

**Introduction Outline for "Living a Good Life in the Age of AI and Automation"**

**1. The Modern Conundrum: Navigating Life with AI and Automation**
   - Introduce the central theme of how the rapid advancement of AI and automation technologies is reshaping the fabric of daily life, influencing everything from work environments to personal relationships.
   - Discuss the dual-edged nature of these technologies, offering unprecedented conveniences and efficiencies on one hand, while presenting new ethical dilemmas and challenges to personal well-being on the other.
   - Use engaging anecdotes or hypothetical scenarios to illustrate how these technologies might impact an average day, from smart homes and AI personal assistants to automated workplaces and social media dynamics.

**2. Philosophical Lenses: Exploring Varied Perspectives**
   - Briefly introduce the eight philosophical schools of thought covered in the book: Stoicism, Epicureanism, Minimalism, Hedonism, Existentialism, Nihilism, Pragmatism, and Idealism.
   - Explain how each philosophy offers unique insights and strategies for dealing with the changes brought about by AI and automation, helping readers to not just cope, but thrive.
   - Highlight the relevance of these ancient and modern philosophies in providing foundational values, ethical frameworks, and personal coping mechanisms in a tech-driven world.

**3. The Dance of Technology and Human Values**
   - Discuss how each philosophical perspective addresses the balance between embracing technological advancements and maintaining essential human values such as privacy, autonomy, and emotional well-being.
   - Explore scenarios where technology might enhance life according to different philosophies, such as Stoicism’s use of tech for self-improvement, or Hedonism’s pursuit of pleasure through immersive virtual realities.
   - Address potential pitfalls and ethical concerns that arise with AI and automation, such as existential risks, job displacement, and the erosion of personal freedoms, viewed through various philosophical prisms.

**4. Practical Wisdom for the Digital Age**
   - Offer readers practical advice drawn from each philosophy on how to navigate the complexities of modern life influenced by AI and automation.
   - Include tips on integrating philosophical practices into daily routines, such as using Minimalism to manage digital clutter, or Pragmatism to assess the real-world impacts of personal tech use.
   - Encourage readers to reflect on their personal values and goals, and how they can use technology as a tool to enhance rather than detract from a fulfilling life.

**5. Embracing the Future with Informed Optimism**
   - Conclude with a forward-looking perspective, emphasizing the role of informed choice and philosophical insight in shaping a future where technology and human values coexist harmoniously.
   - Inspire readers to become proactive participants in the digital age, using the wisdom of the past and present to forge a path that respects both technological potential and the intrinsic worth of human experience.
   - Avoid cliché endings by weaving in a narrative thread that calls back to the introductory anecdotes, demonstrating a full-circle understanding of living well in the age of AI and automation.

This introduction sets the stage for a deep dive into each philosophy in subsequent chapters, preparing the reader to explore how ancient wisdom and modern insights can guide us through the complexities of contemporary life dominated by technology.

Active Reading Progress for Outlines and Chapters for "Living a Good Life in the Age of AI and Automation": 100%|██████████| 1/1 [01:25<00:00, 85.70s/it]       


**Conclusion Outline for "Living a Good Life in the Age of AI and Automation"**

**1. Embracing Philosophical Diversity in the Digital Age**
   - Highlight how each philosophical school of thought, from Stoicism to Pragmatism, offers unique insights into living well amidst rapid technological changes.
   - Discuss the importance of integrating these diverse perspectives to navigate the complexities introduced by AI and automation, such as ethical dilemmas, job displacement, and the redefinition of pleasure and reality.
   - Use examples like the Stoic practice of focusing on what is within one's control to adapt to job automation, or the Epicurean approach to managing desires in a world of endless digital temptations.

**2. The Interplay of Technology and Human Values**
   - Explore how advancements in AI and biotechnology challenge and reshape traditional human values and ethical frameworks.
   - Discuss the role of technology in enhancing human experiences versus the risks it poses, such as dependency and loss of privacy, using insights from Hedonism and Idealism.
   - Illustrate with scenarios where AI enhances human dignity and autonomy, aligned with Idealist principles, versus situations where it might undermine these values.

**3. Practical Wisdom for the Modern World**
   - Offer practical advice drawn from each philosophy on how to live well in an era dominated by digital and automated technologies.
   - For instance, Minimalism's emphasis on decluttering life from unnecessary digital noise, or Pragmatism’s focus on the real-world impacts of technology on human well-being.
   - Provide tips on how to incorporate these philosophical practices into daily life, such as using Stoic exercises to enhance resilience or adopting Nihilistic perspectives to find beauty in simplicity.

**4. Future Visions: Ethical and Philosophical Considerations**
   - Speculate on the future implications of living alongside increasingly intelligent AI, considering both optimistic and cautious viewpoints from the discussed philosophies.
   - Discuss how these philosophical teachings can help shape future policies and personal decisions that ensure technology is used ethically and enhances the human experience.
   - Example: How might Existentialist values influence the design of virtual realities that enhance rather than replace real-world interactions?

**5. The Collective Journey: Building a Thoughtful Society**
   - Conclude by emphasizing the collective role in shaping a future where technology and human values coexist harmoniously.
   - Encourage a global community approach, as suggested by Idealism, to tackle universal challenges posed by AI and automation.
   - Inspire readers to contribute to a society that values deep, philosophical discussions about life’s purpose and the ethical use of technology, fostering a world that respects both individual and collective well-being.

This conclusion will not only summarize the key points from each philosophical perspective but also leave readers with actionable insights and a hopeful vision for integrating these ancient wisdoms into modern life challenges posed by AI and automation.

Active Reading Progress for Outlines and Chapters for "Living a Good Life in the Age of AI and Automation": 100%|██████████| 1/1 [01:18<00:00, 78.62s/it]       


##### Generate actual introduction and conclusion chapters with long-form

In [20]:
introduction_job_id = submit_long_form_job(book_content_kb, introduction_outline)
conclusion_job_id = submit_long_form_job(book_content_kb, conclusion_outline)

##### Edit titles

In [ ]:
patch_chapter_title('Introduction', introduction_job_id)
patch_chapter_title('Conclusion', conclusion_job_id)

# PART 3: USING THE REVISE ENDPOINT -- 2024-04-17

Before exporting to epub, let's use the revise endpoint to change the style of the book.

In [28]:
# for each job id, loop through sections and revise the text
job_ids = [introduction_job_id] + chapter_ids +  [conclusion_job_id]
chapter_markdown_contents = []
for job_id in job_ids:
    # get the job object
    url = f'{SP_BASE_URL}/long_form/{job_id}'
    resp = requests.get(url, auth=API_AUTH)
    chapter = resp.json()['response']

    chapter_markdown = f'# {chapter["title"]}\n\n'

    # loop through sections and revise the text
    for i, section in enumerate(chapter['sections']):
        print(f'REVISING SECTION {i} FOR JOB {job_id}')
        chapter_markdown += f'## {section["title"]}\n\n'
        new_section_text = revise_long_form_section(section['content'], job_id, i)
        chapter_markdown += new_section_text + '\n\nSources:\n\n'

        # add sources if they have links
        for source in section['web_search_results']:
            chapter_markdown += source['url'] + '\n'

        for source in section['ranked_results']:
            if 'link_to_source' in source['metadata']:
                chapter_markdown += source['metadata']['link_to_source'] + '\n'

    chapter_markdown_contents.append(chapter_markdown)

REVISING SECTION 0 FOR JOB cf20accc-92b6-4e0d-aa2f-15338765f1b0


In this new dawn where machines whisper secrets to one another, I find myself caught in the quiet storm of AI and automation. Their tendrils, cold and calculating, weave through the fabric of my daily existence, reshaping the mundane into something almost unrecognizable. I wake to the sound of a mechanical voice, dictating the rhythm of my day—what to eat, how to live, whom to love. There is a sharpness to this convenience, a blade thinly veiled by the promise of efficiency.

My mornings now begin not with the chirping of birds but with the sterile suggestions of an AI assistant, calculating my breakfast with the cold precision of an algorithm. It knows what’s in my pantry, what’s coursing through my veins. Privacy, once my silent companion, now a specter fading into the glow of screens.

At work, the human touch recedes like a distant memory, replaced by the relentless hum of machines that monitor, manage, and manipulate. We are cogs in a grander mechanism, watched over by digital overseers that never blink. Our interactions, stripped of spontaneity, now scripted by software that anticipates every move.

And then, the ever-watchful eyes of social media algorithms, curating my reality, feeding me a world tailored to trap me in an echo chamber. They dictate the news I see, the faces I recognize, the voices I hear. In this digitally ordained solitude, misinformation is the unseen guest at every meal, a specter at every gathering.

As I thread through this labyrinth of code and steel, I find myself grappling with the ghost of freedom, wondering about the essence of connection. The balance between man and machine, a tightrope walk above a digital abyss. In this relentless negotiation with progress, I seek not just to survive but to find a space where my human spirit can still sing amidst the wires and waves. Here, in the shadow of advancing algorithms, I pause and ponder—what does it mean to live, not just exist, in the age of AI?

REVISING SECTION 1 FOR JOB cf20accc-92b6-4e0d-aa2f-15338765f1b0


In this digital dawn, grappling with the echoes of ancient thoughts, I find myself wrestling with shadows—Stoicism, Epicureanism, Minimalism, Hedonism, Existentialism, Nihilism, Pragmatism, Idealism—each whispering old secrets on how to bear the weight of this automated existence. They are ghosts in the machine, their insights flickering like dim lights in vast, unknown corridors of circuitry and code.

Stoicism holds me aloof, a stern mentor teaching restraint and resilience amidst the relentless march of machines. It tells me to sort the chaos of what can be controlled from the relentless surge of what cannot, a beacon against the storm of anxiety stirred by the relentless automation of everything I know. Epicureanism, with its siren song, lures me to bask in the pleasures wrought by technology, to find comfort in the cool touch of AI that simplifies existence, easing the mundane burdens that once tethered my human frailty.

Minimalism cuts through the digital noise, a clear, sharp line drawn in the sand of information overload. It teaches me to strip away the superfluous, to focus on what truly matters even as torrents of data flood through my life. Hedonism, meanwhile, beckons me to dive deep into the sensory cornucopia of virtual realms, to lose myself in the amplified ecstasies that only silicon and screens can provide.

Existentialism, with its fierce grip on personal freedom, urges me to carve my path, to defy the deterministic algorithms that claim to predict my every whim. It challenges me to author my own meaning in this scripted world where choices seem but illusions. Nihilism stares back from the abyss, questioning the very essence of this tech-driven evolution, probing the shadows behind the bright screens that promise endless possibilities.

Pragmatism plants my feet on solid ground, guiding each step based on the tangible impacts of these tools that pervade my days. It demands practical results, a ledger of pros and cons to navigate this wired wilderness. Idealism, ever lofty, dreams of a digital utopia, urging the use of technology to sculpt a fairer society, to elevate collective well-being above the solitary gains of isolated interfaces.

Together, these philosophies form a mosaic of strategies against the relentless tide of automation. They are not just shields to fend off the digital onslaught but also lenses to view a new vista of existence, painting a picture of life woven intricately with threads of silicon and soul. Alone, yet surrounded by the murmurs of these ancient guides, I tread forward, a solitary figure on a digital stage, crafting continuance from the codes of old.

REVISING SECTION 2 FOR JOB cf20accc-92b6-4e0d-aa2f-15338765f1b0


...

In this digital deluge, ancient wisdom whispers through the noise. Minimalism—it screams of stripping back, tearing digital tendrils that choke our minds. It's about what breathes life back into our lungs, what lets us see through the fog of incessant beeps and clicks. This philosophy, it doesn't just whisper; it demands we shred the superfluous, peel away the pixelated layers that don't feed our core.

Pragmatism, now that’s a gritty companion on this pixel-paved path. It weighs the steel of technology in our hands, questions its heft, its intent. It tells us to wield these tools with wary grip, to cut through tasks but beware—lest we cut our ties to flesh and blood. This balance, this dance with digital daemons, pragmatists know it well. They guide our hands, steady our steps.

Then there’s Stoicism, the stoic silence in the storm of data streaming. It teaches us the art of the internal fortress, how to stand firm when waves crash and screens flicker. We journal, we reflect—it's our shield, our armor against the tide of change that threatens to sweep us away. These practices, they anchor us, keep our essence intact in the whirlwind of Wi-Fi wars.

Together, these voices form a chorus, a strategy against the surge of our digitized existence. They teach us to mute the chaos, to tune into a frequency that enhances, not enslaves. Armed with these philosophies, we tread through this techno-terrain with a semblance of grace, ensuring our humanity remains more than just a user ID in the vast, virtual expanse.

REVISING SECTION 3 FOR JOB 6ef27aa4-6048-40e8-a864-44593bca4751


In the looming shadow of AI, entangled in its cold, calculating embrace, the ancient whispers of existentialism echo, urging us to claw back our authenticity in a world slipping into digital dreams. These teachings, they grip our souls, pushing us towards environments where our choices echo our true selves—deep, meaningful realms where virtuality complements reality, nurturing growth, not alienation, in the gardens of our online interactions.

Then, from the void, the stark gaze of Nihilism warns of emptiness, a hollow craving when we lean too heavily on silicon crutches. This bleak vision crafts policies, shaping tools that should serve, not enslave us, ensuring our human essence isn't drowned out by the digital tide. We stand wary of tales that paint AI as a savior, striving instead for balance—a tool wielded with wisdom, sparse and precise.

Ahead, the tangled dance of philosophy and technology could forge paths where ethics frame our digital creations. Drawing from the pragmatism of real-world impacts, we chase advancements that enhance life, not just simplify tasks. We imagine AI that nurtures learning, heals bodies, and respects the sanctity of our private thoughts and choices.

Such futures, as varied as the philosophies guiding them, offer visions through which to explore our coexistence with AI. Grasping these insights, we aim to craft a future where technology respects and amplifies human dignity, weaving efficiency with deep-seated human values, ensuring our journey with AI enriches, not just eases, the human condition. This balanced approach is our beacon in the complex dance with ever-evolving machines, a promise to elevate life's quality, embedding it with dignity and ethical integrity.

REVISING SECTION 4 FOR JOB 6ef27aa4-6048-40e8-a864-44593bca4751


In this dim room, surrounded by the hum of machines, visions of a future dictated by AI and automation haunt me. Driven by Idealism, I try to weave these cold, calculating entities into our human fabric, hoping for a communion deep in thought about life, purpose, and moral machines. This striving reaches beyond today's hurdles, aiming to mold a society that respects the essence of both the one and the many.

Then there's this dream of a world where technology doesn't just serve but understands humanity, a shift from seeing ourselves as mere operators to being integral threads in a vast digital weave. Pondering the expanse of AI, its reach into our culture, our diversity, our very earth, it urges a critical pause—how shall this shape our common future?

Pulling together—governments, businesses, academia, everyday souls—each must play their part in sculpting this tech-driven era to resonate with humanistic tones. Governments dictating the moral compass for technological pursuit, companies pledging honesty in their digital dealings, scholars exploring deep ethical questions, and the vigilant eyes of the public guarding our moral pace.

Here I stand, at the precipice of an era where technology might either uplift or undo us, wrestling with the enormity of the task. By nurturing a global tribe committed to this dual chase, we might yet ensure a digital epoch rich not only in capability but in conscience and soul. This shared commitment could forge a future where technology elevates humanity, enhancing our collective journey and the dignity of each. Through this union, perhaps, technology can truly uplift humanity, enriching our shared existence and individual worth.

##### Save markdown files

In [29]:
fnames = []
for i, chapter_markdown in enumerate(chapter_markdown_contents):
    fname = f'{BOOK_DIRECTORY}/chapter_{i}.md'
    with open(fname, 'w') as f:
        f.write(chapter_markdown)
    fnames.append(fname)

##### Use pandoc to create the book

In [30]:
import subprocess

def generate_epub(output_filename, image_filename):
    # Command parts that are static
    command = [
        'pandoc',
        '-o', output_filename,
        '--toc',
        '--epub-cover-image', image_filename,
        '--metadata', f'title={BOOK_TITLE}',
        '--metadata', f'toc-title={BOOK_TITLE}',
        '--metadata', 'rights=Copyright © 2024 by Superpowered AI. All rights reserved.',
        '--metadata', 'author=Superpowered AI',
    ]

    # Combine the static parts of the command with the filenames
    command.extend(fnames)

    # Execute the command
    try:
        subprocess.run(command, check=True)
        print(f"EPUB successfully created: {output_filename}")
    except subprocess.CalledProcessError as e:
        print(f"Error during EPUB creation: {e}")


generate_epub(f'{BOOK_DIRECTORY}/living_a_good_life.epub', f'{BOOK_DIRECTORY}/living_a_good_life_ai.png')

EPUB successfully created: living_a_good_life/living_a_good_life.epub
